In [1]:
import cv2
import dlib
import faceBlendCommon as fbc
import numpy as np
import math
img = cv2.imread('gettyimages-1200624305-612x612.jpg')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# Read array of corresponding points
# from the faces
points1 = fbc.getLandmarks(detector, predictor, img)

In [2]:
hullIndex = cv2.convexHull(np.array(points1).astype(np.int32), returnPoints=False) 
# Create convex hull lists
hull1 = []
for i in range(0, len(hullIndex)):
    hull1.append(points1[hullIndex[i][0]])
hull_outer = []
for i in range(0, len(hull1)):
    hull_outer.append((hull1[i][0], hull1[i][1]))
mask = np.zeros(img.shape, dtype=img.dtype) 
cv2.fillConvexPoly(mask, np.int32(hull_outer), (255, 255, 255))

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [3]:
tempImg = img.copy()
tempImg = cv2.blur(tempImg,(4,4))
mask_inv = cv2.bitwise_not(mask)
img1_bg = cv2.bitwise_and(img,mask)
img2_fg = cv2.bitwise_and(tempImg,mask_inv)
img_n = cv2.add(img1_bg,img2_fg)

In [4]:
m = cv2.moments(mask[:,:,1])
# a random point chosen
# to calculate radius
k=points1[0]
# Add your Focus cordinates here
# m10/m00 and m01/m00 are the center of the mask 
fx,fy = int(m['m10']/m['m00']), int(m['m01']/m['m00']) 
dist = math. sqrt((k[0]-fx)**2 + (k[1]-fy)**2)
# Standard Deviation of the Gaussian
sigma = dist 
rows,cols = img.shape[:2]
potrait = np.copy(img_n)
potrait[:,:,:] = 0
a = cv2.getGaussianKernel(2*cols ,sigma)[cols-fx:2*cols-fx]
b = cv2.getGaussianKernel(2*rows ,sigma)[rows-fy:2*rows-fy]
c = b*a.T
d = c/c.max()
potrait[:,:,0] = img_n[:,:,0]*d
potrait[:,:,1] = img_n[:,:,1]*d
potrait[:,:,2] = img_n[:,:,2]*d

In [5]:
potrait = cv2.cvtColor(potrait, cv2.COLOR_BGR2GRAY )
cv2.imwrite('potrait_mode.png',potrait)

True